---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [179]:
# Env setup
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:.3f}'.format # suppress scientific notation

In [180]:
# load Energy Indicators.xls
energy = pd.read_excel('Energy Indicators.xls', 
                       header = None,
                       na_values = '...',
                       skiprows = 18,
                       skipfooter = 38,
                       usecols = [2,3,4,5],
                       names = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
                    )

# convert energy from petajoules to gigajoules
energy['Energy Supply'] = energy['Energy Supply'] * 1000000

# country cleanup
energy['Country'] = energy['Country'].str.replace('\d+', '').str.strip()
energy['Country'] = energy['Country'].str.replace(r'\(.*\)','').str.strip()
energy['Country'] = energy['Country'].replace({
    'Republic of Korea': 'South Korea',
    'United States of America': 'United States',
    'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
    'China, Hong Kong Special Administrative Region': 'Hong Kong'})

# set index
# energy = energy.set_index('Country')
energy.head()
# energy.shape (227, 3)

,Country,Energy Supply,Energy Supply per Capita,% Renewable
0,Afghanistan,321000000.000,10.000,78.669
1,Albania,102000000.000,35.000,100.000
2,Algeria,1959000000.000,51.000,0.551
3,American Samoa,nan,nan,0.641
4,Andorra,9000000.000,121.000,88.696


In [181]:
# load world_bank.csv, GDP data
GDP = pd.read_csv('world_bank.csv', skiprows = 4, header = 0)

# country cleanup
GDP['Country Name'] = GDP['Country Name'].replace({
    'Korea, Rep.': 'South Korea', 
    'Iran, Islamic Rep.': 'Iran',
    'Hong Kong SAR, China': 'Hong Kong'})

# drop some columns
GDP = GDP[['Country Name', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015']]
# GDP.columns

# set the index
GDP = GDP.rename(columns={'Country Name': 'Country'})
# GDP = GDP.set_index('Country')

GDP.head()
# GDP.shape (264, 10)

,Country,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Aruba,nan,nan,nan,nan,2467703910.615,nan,nan,nan,nan,nan
1,Andorra,4018195653.510,4021331428.401,3675727910.507,3535389132.602,3346317328.525,3185604581.520,3129537611.886,3127549907.584,nan,nan
2,Afghanistan,10305228125.139,11721187594.205,12144482858.180,14697331940.646,15936800636.249,16911126453.228,19352203805.629,19731337260.638,19990317160.676,20294152243.007
3,Angola,55811028114.648,68420444001.499,77874196512.184,79753199398.140,82470913120.731,85702615836.440,90120963241.219,96261432368.264,100886286610.958,103910558934.020
4,Albania,9771760095.126,10348293941.585,11127520474.298,11500292411.174,11926953258.916,12231090567.667,12404772053.501,12542465022.373,12793314322.820,13120823169.485


In [182]:
# load scimagojr-3.xlsx, ranking based on journal's citations
ScimEn = pd.read_excel('scimagojr-3.xlsx')
ScimEn2 = ScimEn.copy()
# subsetting
ScimEn = ScimEn[ScimEn['Rank'] < 16]

# set the index
# ScimEn = ScimEn.set_index('Country')
# ScimEn2 = ScimEn2.set_index('Country')

ScimEn.head()
ScimEn2.head()

ScimEn.shape # (15,7)
ScimEn2.shape # (191, 7)

(191, 8)

In [183]:
# join energy, GDP and ScimEn by Country Name
q1_df = pd.merge(ScimEn, energy, how = 'inner', on = 'Country')
q1_df = pd.merge(q1_df, GDP, how = 'inner', on = 'Country')

# set index to Country
q1_df = q1_df.set_index('Country')

q1_df.shape
q1_df

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.700,138,127191000000.000,93.000,19.755,3992331462570.890,4559041383517.930,4997775145605.960,5459247267275.140,6039658508485.590,6612490294273.650,7124977970399.610,7672447738017.420,8230120617793.520,8797998940421.279
United States,2,96661,94747,792274,265436,8.200,230,90838000000.000,286.000,11.571,14792303791800.000,15055395304800.000,15011490541400.000,14594842181900.000,14964372000000.000,15204019634600.000,15542161722300.000,15773666098800.000,16156618476000.000,16548573278300.000
Japan,3,30504,30287,223024,61554,7.310,134,18984000000.000,149.000,10.233,5496541517363.310,5617035943770.730,5558526873313.820,5251308399909.320,5498717815809.770,5473738114222.050,5569101602926.180,5644658523313.420,5642884448582.190,5669563143140.970
United Kingdom,4,20944,20357,206091,37874,9.840,139,7920000000.000,124.000,10.600,2419630700401.730,2482203235475.900,2470614405284.300,2367047611711.990,2403504326328.800,2450911011124.850,2479808627472.190,2533370110012.360,2605643189894.930,2666333396477.130
Russian Federation,5,18534,18301,34266,12422,1.850,57,30709000000.000,214.000,17.289,1385792607182.580,1504071117741.300,1583004071092.110,1459199142720.260,1524917468442.010,1589942641764.290,1645875899596.270,1666934123163.160,1678708855070.860,1616148858033.770
Canada,6,17899,17620,215003,40930,12.010,149,10431000000.000,296.000,61.945,1564468853329.450,1596739898466.320,1612713061250.240,1565144676082.310,1613406134731.120,1664086754804.890,1693132927878.080,1730687770724.130,1773485821199.770,1792608764608.810
Germany,7,17027,16831,140566,27426,8.260,126,13261000000.000,165.000,17.902,3332890754966.890,3441560834437.090,3478809377483.440,3283339933774.830,3417298013245.030,3542371125827.810,3556723774834.440,3567317390728.480,3624386278145.700,3685555907284.770
India,8,15005,14841,128763,37209,8.580,115,33195000000.000,26.000,14.969,1265894322131.090,1374865195244.500,1428360609657.800,1549482541544.700,1708458876829.920,1821872415623.620,1924235460945.250,2051981849791.690,2200616575310.970,2367206118959.060
France,9,13153,12973,130632,28601,9.930,114,10597000000.000,166.000,17.020,2607839735099.340,2669423841059.600,2674637086092.710,2595966887417.220,2646994701986.750,2702031788079.470,2706968211920.530,2722566887417.220,2729631788079.470,2761185430463.580


In [184]:
def answer_one():
    return q1_df

# answer_one().shape
# q1_df.shape #(15,20)

### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [185]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [186]:
# outer join
q2_df_outer = pd.merge(ScimEn2, energy, how = 'outer', on = 'Country')
q2_df_outer = pd.merge(q2_df_outer, GDP, how = 'outer', on = 'Country')

q2_df_outer['Country'].head()
q2_df_outer.shape # (318, 21)

# inner join
q2_df_inner = pd.merge(ScimEn2, energy, how = 'inner', on = 'Country')
q2_df_inner = pd.merge(q2_df_inner, GDP, how = 'inner', on = 'Country')

q2_df_inner['Country'].head()
q2_df_inner.shape # (162, 21)

def answer_two():
    return (q2_df_outer.shape[0] - q2_df_inner.shape[0])

# answer.two()
# 156

## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [187]:
# define columns to be used for the calculation of GDP mean value per country
cols_for_mean = ['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
avgGDP = q1_df.apply(lambda x: np.mean(x[cols_for_mean]), axis=1)

def answer_three():
    Top15 = answer_one()
    return avgGDP.sort_values(ascending = False)

# answer_three()

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [188]:
# it should be the absolute change based on forum

def answer_four():
    Top15 = answer_one()
    return q1_df.loc['United Kingdom']['2015'] - q1_df.loc['United Kingdom']['2006']

# answer_four()
# 246702696075.3999

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [189]:
def answer_five():
    Top15 = answer_one()
    return q1_df['Energy Supply per Capita'].mean()

# answer_five()
# 157.6

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [190]:
q6_pct = q1_df['% Renewable'].max() # 69.648
q6_country = q1_df['% Renewable'].idxmax() # 'Brazil'

q6_tuple = ()

def answer_six():
    Top15 = answer_one()
    return q6_tuple + (q6_country, q6_pct)

# answer_six()
# type(answer_six())
# ('Brazil', 69.648030000000006)

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [191]:
# make a copy of df so as not to change the dimensions of df for answer_one()
q1_df_copy = q1_df.copy()
q1_df_copy['Citations_ratio'] = q1_df_copy['Self-citations'] / q1_df_copy['Citations']
q7_country = q1_df_copy['Citations_ratio'].idxmax()
q7_pct = q1_df_copy['Citations_ratio'].max()
q7_tuple = ()

def answer_seven():
    Top15 = answer_one()
    return q7_tuple + (q7_country, q7_pct)

# answer_seven()
# type(answer_seven())
# ('China', 0.68931261793894216)

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [192]:
q1_df_copy['Population_estimate'] = q1_df_copy['Energy Supply'] / q1_df_copy['Energy Supply per Capita']
q8_df = q1_df_copy['Population_estimate'].sort_values(ascending = False)

def answer_eight():
    Top15 = answer_one()
    return q8_df.index[2] 

# answer_eight()
# 'United States'

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [193]:
q1_df_copy['Citable_docs_per_person'] = q1_df_copy['Citable documents'] / q1_df_copy['Population_estimate']


def answer_nine():
    Top15 = answer_one()
    return q1_df_copy['Citable_docs_per_person'].corr(q1_df_copy['Energy Supply per Capita'])

# answer_nine()
# answer_nine().dtype
# 0.79400104354429424

In [194]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    q1_df_copy = answer_one()
    q1_df_copy['Population_estimate'] = q1_df_copy['Energy Supply'] / q1_df_copy['Energy Supply per Capita']
    q1_df_copy['Citable_docs_per_person'] = q1_df_copy['Citable documents'] / q1_df_copy['Population_estimate']
    q1_df_copy.plot(x='Citable_docs_per_person', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [195]:
# plot9() # Be sure to comment out plot9() before submitting the assignment!

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [196]:
median = q1_df_copy['% Renewable'].median() # 17.02028
q1_df_copy['Renewable_flg'] = np.where(q1_df_copy['% Renewable'] >= median, 1, 0)
HighRenew = q1_df_copy['Renewable_flg']

def answer_ten():
    Top15 = answer_one()
    return HighRenew

# type(answer_ten())
# answer_ten()

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [197]:
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}

q11_df = q1_df_copy.groupby(ContinentDict)['Population_estimate'].agg(['size','sum','mean','std'])
q11_df.index.name = 'Continent'

def answer_eleven():
    Top15 = answer_one()
    return q11_df

# answer_eleven()
# type(answer_eleven()) # pandas.core.frame.DataFrame

### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [198]:
bins = pd.cut(q1_df_copy['% Renewable'], 5)
q12_df = q1_df_copy.groupby([ContinentDict, bins]).size()
q12_df.index.names = (['Continent', '% Renewable'])

def answer_twelve():
    Top15 = answer_one()
    return q12_df

# answer_twelve()
# type(answer_twelve()) # pandas.core.series.Series

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [199]:
q13_df = q1_df_copy['Population_estimate'].map('{:,}'.format)
q13_df.name = 'PopEst'

def answer_thirteen():
    Top15 = answer_one()
    return q13_df

# answer_thirteen()
# type(answer_thirteen())
# type(answer_thirteen().iloc[0])

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [200]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [201]:
# plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!